In [4]:
# if used pip install package
# !pip install xgboost
# !pip install lightgbm
# !pip install wget

In [100]:
import os
import sys
import wget
import time
import tarfile
import zipfile
import random

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
import lightgbm as lgb


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
DDIR = "data"
UDDIR = "user_data"
RESDIR = "prediction_result"

In [7]:
UID = "user_id"

# Load data (Only once)

In [8]:
# train_fname = wget.download("https://tesla-ap-shanghai-1256322946.cos.ap-shanghai.myqcloud.com/cephfs/tesla_common/deeplearning/dataset/algo_contest/train_preliminary.zip", out=DDIR)
# test_fname = wget.download("https://tesla-ap-shanghai-1256322946.cos.ap-shanghai.myqcloud.com/cephfs/tesla_common/deeplearning/dataset/algo_contest/test.zip", out=DDIR)

In [9]:
# def myunzip(filename):
#     zFile = zipfile.ZipFile(filename, "r")
#     for fileM in zFile.namelist(): 
#         zFile.extract(fileM, DDIR)
#         print(fileM)
#     zFile.close()

In [10]:
# myunzip(train_fname)
# myunzip(test_fname)

# Utils

In [11]:
def bch_rencol(values, prefix="", suffix=""):
    return list(map(lambda x: f"{prefix}"+"_".join(list(map(lambda y: str(y), x)))+f"{suffix}" 
                    if hasattr(x, "__iter__") and not isinstance(x, str) 
                    else f"{prefix}"+str(x)+f"{suffix}", values))

In [156]:
def mynunique(values):
    return values.nunique(dropna=False)

# Data Exploration

In [12]:
# read train data
train_click_log = pd.read_csv(f"{DDIR}/train_preliminary/click_log.csv")
train_ad = pd.read_csv(f"{DDIR}/train_preliminary/ad.csv")
# tag
train_user = pd.read_csv(f"{DDIR}/train_preliminary/user.csv")

In [13]:
# read test data
test_click_log = pd.read_csv(f"{DDIR}/test/click_log.csv")
test_ad = pd.read_csv(f"{DDIR}/test/ad.csv")

In [14]:
train_click_log.head()

,time,user_id,creative_id,click_times
0,9,30920,567330,1
1,65,30920,3072255,1
2,56,30920,2361327,1
3,6,309204,325532,1
4,59,309204,2746730,1


In [15]:
train_ad.head()

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,\N,5,381,78
1,4,4,\N,5,108,202
2,7,7,\N,5,148,297
3,8,8,\N,5,713,213
4,9,9,\N,5,695,213


In [16]:
test_click_log.head()

,time,user_id,creative_id,click_times
0,20,3131989,645764,1
1,20,3131989,1027422,1
2,20,3131989,1106443,1
3,20,3131989,629802,1
4,59,3131989,2839769,1


In [17]:
test_ad.head()

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,\N,5,381,78
1,3,3,\N,5,598,165
2,4,4,\N,5,108,202
3,5,5,\N,5,109,6
4,6,6,\N,5,400,6


In [18]:
train_user.head()

,user_id,age,gender
0,1,4,1
1,2,10,1
2,3,7,2
3,4,5,1
4,5,4,1


In [19]:
train_click_log.head()

,time,user_id,creative_id,click_times
0,9,30920,567330,1
1,65,30920,3072255,1
2,56,30920,2361327,1
3,6,309204,325532,1
4,59,309204,2746730,1


In [20]:
train_ad.head()

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,\N,5,381,78
1,4,4,\N,5,108,202
2,7,7,\N,5,148,297
3,8,8,\N,5,713,213
4,9,9,\N,5,695,213


In [21]:
train_ad.creative_id.nunique()

2481135

In [22]:
train_ad.ad_id.nunique()

2264190

In [134]:
train_ad["product_id"][0] == "\\N"

True

In [141]:
train_ad["product_id"] = train_ad["product_id"].replace({"\\N": np.nan})
test_ad["product_id"] = test_ad["product_id"].replace({"\\N": np.nan})

# Feature engineering

In [161]:
train_feat = pd.DataFrame()
test_feat = pd.DataFrame()

In [162]:
train_feat[UID] = train_user[UID]
test_feat[UID] = np.sort(test_click_log[UID].unique())

In [152]:
tol_train = pd.merge(train_click_log, train_ad, how="left", on="creative_id")
tol_test = pd.merge(test_click_log, test_ad, how="left", on="creative_id")

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,9,30920,567330,1,504423,30673,3,32638,319
1,65,30920,3072255,1,2642300,1261,2,6783,6
2,56,30920,2361327,1,2035918,1261,2,6783,6
3,6,309204,325532,1,292523,27081,3,32066,242
4,59,309204,2746730,1,2362208,NaN,18,14682,88


## User Click log

In [155]:
tol_train.head()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,9,30920,567330,1,504423,30673,3,32638,319
1,65,30920,3072255,1,2642300,1261,2,6783,6
2,56,30920,2361327,1,2035918,1261,2,6783,6
3,6,309204,325532,1,292523,27081,3,32066,242
4,59,309204,2746730,1,2362208,NaN,18,14682,88


In [127]:
train_click_log.shape

(30082771, 4)

In [160]:
# for train
tmp = tol_train.groupby([UID]).agg(
    {
        UID: ["count"], 
        "click_times": ["sum", "max", "mean", "std"],
        "time": [mynunique, "mean", "max", "min"],
        "creative_id": [mynunique],
        "ad_id": [mynunique],
        "product_id": [mynunique],
        "product_category": [mynunique],
        "advertiser_id": [mynunique],
        "industry": [mynunique],
    }
)

In [165]:
tmp.columns = bch_rencol(tmp.columns)
tmp.reset_index(inplace=True)
train_feat = pd.merge(train_feat, tmp, how="left", on=UID)

In [166]:
tmp = None

In [167]:
# for test
tmp = tol_test.groupby([UID]).agg(
    {
        UID: ["count"], 
        "click_times": ["sum", "max", "mean", "std"],
        "time": [mynunique, "mean", "max", "min"],
        "creative_id": [mynunique],
        "ad_id": [mynunique],
        "product_id": [mynunique],
        "product_category": [mynunique],
        "advertiser_id": [mynunique],
        "industry": [mynunique],
    }
)

In [169]:
tmp.columns = bch_rencol(tmp.columns)
tmp.reset_index(inplace=True)
test_feat = pd.merge(test_feat, tmp, how="left", on=UID)

In [170]:
tmp = None

In [171]:
train_all = pd.merge(train_feat, train_user, how="left", on=UID)

In [172]:
train_tag = train_all[[UID, "age", "gender"]]
train_feat = train_all.drop(["age", "gender"], axis=1)

# Training&Prediction

In [173]:
# split data
train_feat_tr, train_feat_val, train_tag_tr, train_tag_val = train_test_split(train_feat, train_tag, test_size=0.2)

## For Age

In [176]:
lgbds_train_tr_age = lgb.Dataset(train_feat_tr, train_tag_tr["age"]-1)
lgbds_train_val_age = lgb.Dataset(train_feat_val, train_tag_val["age"]-1)

In [177]:
params_age = {
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "num_class": 10,
    "metric": "multi_error",
    "learning_rate": 0.01,
}

In [178]:
model_lgb_multi_age = lgb.train(params_age, lgbds_train_tr_age, num_boost_round=1000, valid_sets=[lgbds_train_val_age], verbose_eval=50, early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds
[50]	valid_0's multi_error: 0.774933
[100]	valid_0's multi_error: 0.77375
[150]	valid_0's multi_error: 0.773083
[200]	valid_0's multi_error: 0.772683
[250]	valid_0's multi_error: 0.772456
[300]	valid_0's multi_error: 0.7725
[350]	valid_0's multi_error: 0.7723
[400]	valid_0's multi_error: 0.772167
[450]	valid_0's multi_error: 0.772161
[500]	valid_0's multi_error: 0.77235
Early stopping, best iteration is:
[431]	valid_0's multi_error: 0.772094


In [ ]:
# offline

In [180]:
train_val_age_prob = model_lgb_multi_age.predict(train_feat_val, num_iteration=model_lgb_multi_age.best_iteration)

In [181]:
train_val_age_pred = [list(x).index(max(x))+1 for x in train_val_age_prob]

In [182]:
accuracy_score(train_val_age_pred, train_tag_val["age"])

0.22790555555555556

In [192]:
0.6746444444444445+0.22790555555555556

0.9025500000000001

## For Gender

In [183]:
lgbds_train_tr_gender = lgb.Dataset(train_feat_tr, train_tag_tr["gender"]-1)
lgbds_train_val_gender = lgb.Dataset(train_feat_val, train_tag_val["gender"]-1)

In [186]:
params_gender = {
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "num_class": 2,
    "metric": "multi_error",
    "learning_rate": 0.01,
}

In [187]:
model_lgb_multi_gender = lgb.train(params_gender, lgbds_train_tr_gender, num_boost_round=1000, valid_sets=[lgbds_train_val_gender], verbose_eval=50, early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds
[50]	valid_0's multi_error: 0.32915
[100]	valid_0's multi_error: 0.328067
[150]	valid_0's multi_error: 0.327733
[200]	valid_0's multi_error: 0.327467
[250]	valid_0's multi_error: 0.327306
[300]	valid_0's multi_error: 0.327078
[350]	valid_0's multi_error: 0.327028
[400]	valid_0's multi_error: 0.326911
[450]	valid_0's multi_error: 0.326789
[500]	valid_0's multi_error: 0.326561
[550]	valid_0's multi_error: 0.326339
[600]	valid_0's multi_error: 0.326267
[650]	valid_0's multi_error: 0.326078
[700]	valid_0's multi_error: 0.325906
[750]	valid_0's multi_error: 0.325789
[800]	valid_0's multi_error: 0.325706
[850]	valid_0's multi_error: 0.325789
[900]	valid_0's multi_error: 0.325556
[950]	valid_0's multi_error: 0.325478
[1000]	valid_0's multi_error: 0.325372
Did not meet early stopping. Best iteration is:
[990]	valid_0's multi_error: 0.325356


In [ ]:
# offline

In [188]:
train_val_gender_prob = model_lgb_multi_gender.predict(train_feat_val, num_iteration=model_lgb_multi_gender.best_iteration)

In [189]:
train_val_gender_pred = [list(x).index(max(x))+1 for x in train_val_gender_prob]

In [190]:
accuracy_score(train_val_gender_pred, train_tag_val["gender"])

0.6746444444444445

# Generate Prediction Result

In [193]:
res = test_feat[[UID]]

In [194]:
res["predicted_age"] = [list(x).index(max(x))+1 for x in model_lgb_multi_age.predict(test_feat, num_iteration=model_lgb_multi_age.best_iteration)]

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [195]:
res["predicted_gender"] = [list(x).index(max(x))+1 for x in model_lgb_multi_gender.predict(test_feat, num_iteration=model_lgb_multi_gender.best_iteration)]

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [220]:
res_suffix = time.strftime("%Y%m%d%H%M%S", time.localtime(int(time.time())))
res.to_csv(f"{RESDIR}/res-{res_suffix}.csv", index=False)

In [219]:
res["predicted_age"] = np.round([10 if x > 10 else x for x in res["predicted_age"] + np.random.rand(len(res))], 3)
res["predicted_gender"] = np.round([2 if x > 2 else x for x in res["predicted_gender"] + np.random.rand(len(res))], 3)

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\program files\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Cent result to COS

In [24]:
from ti import session
ti_session = session.Session()

In [63]:
inputs = ti_session.upload_data(path=f"{RESDIR}/init_res.csv", bucket="etveritas-1252104022", key_prefix=RESDIR)